# Market Learning
*Leveraging machine learning to help you learn the market and invest your money*

### Outline
- Objective
- Data Collection
- Data Cleaning
- Visualization
- Prediction
- Conclusion

### Objective
The Market Learning project is intended to provide an adaptive, fact-based introduction of the stock market to a user with limited experience looking for help with personal investment decisions.

### Data Collection
Collect data from sources
